In [1]:
from Code.UtilityFunctions.run_query import run_query

### How many people live in the city of New York?

In [ ]:
sparql_query = """

SELECT ?population ?date
WHERE {
    wdt:Q60 wdt:P1082 ?population . #new york population
    ?population wdt:P585 ?date # point in time
}
SORT BY DESC(?date)
LIMIT 1
"""

run_query(query=sparql_query, as_dataframe=True)

### How many people live in the state of Maryland?

In [ ]:
sparql_query = """

SELECT ?population ?date
WHERE {
    wdt:Q1391 wdt:P1082 ?population . #maryland population
    ?population wdt:P585 ?date # point in time
}
SORT BY DESC(?date)
LIMIT 1
"""

run_query(query=sparql_query, as_dataframe=True)

### What drinks exist if I want a hot beverage?

In [ ]:
sparql_query = """

SELECT ?drink
WHERE {
    ?drink wdt:P279 wdt:Q19359564 # subclass of hot beverage
}
"""

run_query(query=sparql_query, as_dataframe=True)

### How many shops does Target have in North America?

### How many cities are there in California?

In [ ]:
sparql_query = """

SELECT DISTINCT COUNT(?city)
WHERE {
    ?city wdt:P131 wdt:Q99 . # located in the administrative territorial entity California
}
"""

run_query(query=sparql_query, as_dataframe=True)

### How many cities are not part of the USA?

In [ ]:
sparql_query = """

SELECT DISTINCT COUNT(?city)
WHERE {
    ?city wdt:P17 ?country . # country
    FILTER NOT EXISTS {?city wdt:P17 wdt:Q30 } #country USA
}
"""

run_query(query=sparql_query, as_dataframe=True)